In [ ]:
# %pip install pycirclizely

In [2]:
import plotly.io as pio
from IPython.display import HTML

# Circos plot: eukaryotes

In [3]:
from Bio.SeqFeature import SeqFeature


def hover_formatter(feature: SeqFeature) -> str:
    """Custom function for genomic features annotation hover text."""
    info = []

    if hasattr(feature, "qualifiers") and feature.qualifiers.get("gene", [""])[0]:
        info.append(f"Gene: {feature.qualifiers['gene'][0]}")

    if hasattr(feature, "type") and feature.type:
        info.append(f"Type: {feature.type}")

    if hasattr(feature, "location") and hasattr(feature.location, "strand"):
        strand = "+" if feature.location.strand == 1 else "-"
        info.append(f"Strand: {strand}")

    if hasattr(feature, "location"):
        start = (
            int(feature.location.start) if hasattr(feature.location, "start") else "?"
        )
        end = int(feature.location.end) if hasattr(feature.location, "end") else "?"
        info.append(f"Position: {start}-{end}")

    if hasattr(feature, "qualifiers") and feature.qualifiers.get("product", [""])[0]:
        info.append(f"Product: {feature.qualifiers['product'][0]}")

    return "<br>".join(info) if info else "No information available"

## 4 Homo sapiens (hg38)

**hg38** data files are obtained from [UCSC Table Browser](https://genome.ucsc.edu/cgi-bin/hgTables).

Dataset Repository:  
https://github.com/moshi4/pycirclizely-data/tree/main/eukaryote/hg38

### 4-1. Ideograms

In [ ]:
from pycirclizely import Circos
from pycirclizely.utils import load_eukaryote_example_dataset

# Load hg38 dataset (https://github.com/moshi4/pycirclizely-data/tree/main/eukaryote/hg38)
chr_bed_file, cytoband_file, _ = load_eukaryote_example_dataset("hg38")

# Initialize Circos from BED chromosomes
circos = Circos.initialize_from_bed(chr_bed_file, space=3)
circos.text("Homo sapiens (hg38)", font=dict(size=15))

# Add cytoband tracks from cytoband file
circos.add_cytoband_tracks((95, 100), cytoband_file, show_hovertext=True)

# Plot chromosome name
for sector in circos.sectors:
    sector.text(sector.name, font=dict(size=12))

fig = circos.plotfig()
HTML(pio.to_html(fig, include_plotlyjs="cdn"))

### 4-2. Segmental Dups Link

In [ ]:
from pycirclizely import Circos
from pycirclizely.utils import ColorCycler, load_eukaryote_example_dataset

# Load hg38 dataset (https://github.com/moshi4/pycirclizely-data/tree/main/eukaryote/hg38)
chr_bed_file, cytoband_file, chr_links = load_eukaryote_example_dataset("hg38")

# Initialize Circos from BED chromosomes
circos = Circos.initialize_from_bed(chr_bed_file, space=3)
circos.text("Homo sapiens<br>(hg38)", deg=315, r=150, font=dict(size=12))

# Add cytoband tracks from cytoband file
circos.add_cytoband_tracks((95, 100), cytoband_file)

# Create chromosome color dict
color_cycler = ColorCycler("Rainbow")
chr_names = [s.name for s in circos.sectors]
colors = color_cycler.get_colors(len(chr_names))[::-1]
chr_name2color = {name: color for name, color in zip(chr_names, colors)}

# Plot chromosome name & xticks
for sector in circos.sectors:
    sector.text(
        sector.name, r=120, font=dict(size=10, color=chr_name2color[sector.name])
    )
    sector.get_track("cytoband").xticks_by_interval(
        40000000,
        label_orientation="vertical",
        label_formatter=lambda v: f"{v / 1000000:.0f} Mb",
        text_kws=dict(font=dict(size=8)),
    )

# Plot chromosome link
for link in chr_links:
    region1 = (link.query_chr, link.query_start, link.query_end)
    region2 = (link.ref_chr, link.ref_start, link.ref_end)
    color = chr_name2color[link.query_chr]
    if link.query_chr in ("chr1", "chr8", "chr16") and link.query_chr != link.ref_chr:
        circos.link(
            region1, region2, fillcolor=color, line=dict(color=color, width=0.1)
        )

fig = circos.plotfig()
HTML(pio.to_html(fig, include_plotlyjs="cdn"))

### 4-3. Plot Graphs

In [ ]:
import numpy as np

from pycirclizely import Circos
from pycirclizely.utils import ColorCycler, load_eukaryote_example_dataset

np.random.seed(0)

# Load hg38 dataset (https://github.com/moshi4/pycirclizely-data/tree/main/eukaryote/hg38)
chr_bed_file = load_eukaryote_example_dataset("hg38")[0]

# Initialize Circos from BED chromosomes
circos = Circos.initialize_from_bed(
    chr_bed_file, start=-80, end=260, space=3, endspace=False
)
circos.text("Homo sapiens<br>(hg38)", font=dict(size=15))

# Create chromosome color dict
color_cycler = ColorCycler("Rainbow")
chr_names = [s.name for s in circos.sectors]
colors = color_cycler.get_colors(len(chr_names))[::-1]
chr_name2color = {name: color for name, color in zip(chr_names, colors)}

for sector in circos.sectors:
    # Plot chromosome outer track
    sector.text(sector.name.replace("chr", ""))
    color = chr_name2color[sector.name]
    outer_track = sector.add_track((95, 100))
    outer_track.axis(fillcolor=color)
    # Create example x,y plot data
    step = 10000000
    x = np.arange(sector.start + (step / 2), sector.end - (step / 2), step)
    y = np.random.randint(0, 100, size=len(x))
    # Scatter track
    track1 = sector.add_track((80, 90), r_pad_ratio=0.1)
    track1.axis()
    track1.scatter(x, y, vmax=100, marker=dict(size=3, color="red"))
    # Line track
    track2 = sector.add_track((65, 75), r_pad_ratio=0.1)
    track2.axis()
    track2.line(x, y, vmax=100, line=dict(color="blue"))
    # Bar track
    track3 = sector.add_track((50, 60), r_pad_ratio=0.1)
    track3.axis()
    track3.bar(x, y, vmax=100, width=step * 0.7, fillcolor="olive")
    # Fill between track
    track4 = sector.add_track((35, 45), r_pad_ratio=0.1)
    track4.axis()
    track4.grid()
    track4.fill_between(x, y, vmax=100, fillcolor="violet")
    # Plot track labels
    if sector.name == circos.sectors[0].name:
        circos.text("Chr", r=outer_track.r_center, deg=-90)
        circos.text("a", r=track1.r_center, deg=-90, font=dict(color="red"))
        circos.text("b", r=track2.r_center, deg=-90, font=dict(color="blue"))
        circos.text("c", r=track3.r_center, deg=-90, font=dict(color="olive"))
        circos.text("d", r=track4.r_center, deg=-90, font=dict(color="violet"))

fig = circos.plotfig()
HTML(pio.to_html(fig, include_plotlyjs="cdn"))

## 5 Mus musculus (mm10)

**mm10** data files are obtained from [UCSC Table Browser](https://genome.ucsc.edu/cgi-bin/hgTables).

Dataset Repository:  
https://github.com/moshi4/pycirclizely-data/tree/main/eukaryote/mm10

### 5-1. Ideograms

In [ ]:
from pycirclizely import Circos
from pycirclizely.utils import load_eukaryote_example_dataset

# Load mm10 dataset (https://github.com/moshi4/pycirclizely-data/tree/main/eukaryote/mm10)
chr_bed_file, cytoband_file, _ = load_eukaryote_example_dataset("mm10")

# Initialize Circos from BED chromosomes
circos = Circos.initialize_from_bed(chr_bed_file, space=3)
circos.text("Mus musculus (mm10)", font=dict(size=15))

# Add cytoband tracks from cytoband file
circos.add_cytoband_tracks((95, 100), cytoband_file, show_hovertext=True)

# Plot chromosome name
for sector in circos.sectors:
    sector.text(sector.name, font=dict(size=10))

fig = circos.plotfig()
HTML(pio.to_html(fig, include_plotlyjs="cdn"))

### 5-2. Segmental Dups Link

In [ ]:
from pycirclizely import Circos
from pycirclizely.utils import ColorCycler, load_eukaryote_example_dataset

# Load mm10 dataset (https://github.com/moshi4/pycirclizely-data/tree/main/eukaryote/mm10)
chr_bed_file, cytoband_file, chr_links = load_eukaryote_example_dataset("mm10")

# Initialize Circos from BED chromosomes
circos = Circos.initialize_from_bed(chr_bed_file, space=3)
circos.text("Mus musculus<br>(mm10)", deg=315, r=150, font=dict(size=12))

# Add cytoband tracks from cytoband file
circos.add_cytoband_tracks((95, 100), cytoband_file)

# Create chromosome color dict
color_cycler = ColorCycler("Rainbow")
chr_names = [s.name for s in circos.sectors]
colors = color_cycler.get_colors(len(chr_names))[::-1]
chr_name2color = {name: color for name, color in zip(chr_names, colors)}

# Plot chromosome name & xticks
for sector in circos.sectors:
    sector.text(
        sector.name, r=120, font=dict(size=10, color=chr_name2color[sector.name])
    )
    sector.get_track("cytoband").xticks_by_interval(
        50000000,
        label_orientation="vertical",
        label_formatter=lambda v: f"{v / 1000000:.0f} Mb",
        text_kws=dict(font=dict(size=8)),
    )

# Plot chromosome link
for link in chr_links:
    region1 = (link.query_chr, link.query_start, link.query_end)
    region2 = (link.ref_chr, link.ref_start, link.ref_end)
    color = chr_name2color[link.query_chr]
    if link.query_chr != link.ref_chr:
        circos.link(
            region1, region2, fillcolor=color, line=dict(color=color, width=0.1)
        )

fig = circos.plotfig()
HTML(pio.to_html(fig, include_plotlyjs="cdn"))

### 5-3. Plot Heatmaps

In [ ]:
import numpy as np

from pycirclizely import Circos
from pycirclizely.utils import ColorCycler, load_eukaryote_example_dataset

np.random.seed(0)

# Load mm10 dataset (https://github.com/moshi4/pycirclizely-data/tree/main/eukaryote/mm10)
chr_bed_file = load_eukaryote_example_dataset("mm10")[0]

# Initialize Circos from BED chromosomes
circos = Circos.initialize_from_bed(
    chr_bed_file, space=2, start=10, end=350, endspace=False
)
circos.text("Mus musculus<br>(mm10)", font=dict(size=16))

heatmap_cmaps = [
    "RdBu_r",
    "Spectral",
    "Viridis",
    "Plasma",
    "Reds",
    "Blues",
    "Greens",
    "Greys",
]
color_cycler = ColorCycler("Rainbow")
colors = color_cycler.get_colors(count=len(circos.sectors))[::-1]
for sector, color in zip(circos.sectors, colors):
    sector.text(sector.name.replace("chr", ""), font=dict(size=12))
    # Plot chromosome outer track
    chr_track = sector.add_track((95, 100))
    chr_track.axis(fillcolor=color)
    # Create random test data for heatmap plot
    window_size = 10_000_000
    data_num = int(sector.size // window_size) + 1
    vmin, vmax = 0, 100
    data = np.random.randint(vmin, vmax, data_num)
    # Plot heatmap tracks with various cmap
    track_r_size, r_interval, r_start = 4, 2, 90
    for idx, cmap in enumerate(heatmap_cmaps):
        r_pos = r_start - (track_r_size + r_interval) * idx
        track_r_lim = (r_pos - track_r_size, r_pos)
        track = sector.add_track(track_r_lim)
        track.axis(line=dict(color="grey"))
        track.heatmap(
            data,
            width=window_size,
            vmin=vmin,
            vmax=vmax,
            cmap=cmap,
            hover_text=None,
            rect_kws=dict(line=dict(width=0.0)),
        )
        # Plot colormap name on center
        if sector.name == circos.sectors[0].name:
            circos.text(cmap, r=track.r_center, font=dict(size=10))

fig = circos.plotfig()
HTML(pio.to_html(fig, include_plotlyjs="cdn"))